In [1]:
# !pip install pronouncing

In [2]:
# !pip install Metaphone

  Preparing metadata (setup.py) ... done
  Created wheel for Metaphone: filename=Metaphone-0.6-py3-none-any.whl size=13902 sha256=018fc34d9085ec7a233d64dee2185133aeeb18655454a66ca82ce9bf5d9f2cc4
  Stored in directory: /home/byczong/.cache/pip/wheels/23/dd/1d/6cdd346605db62bde1f60954155e9ce48f4681c243f265b704
Successfully built Metaphone


In [1]:
import pronouncing
from metaphone import doublemetaphone
import re

### Finding rhymes

- Goal: Find set of words (unigrams) that rhyme with a given word

- Rough algorithm plan:
    - Input: `s`: str
    - Expand any abbreviations, symbols and numbers in `s` ('mr' -> 'mister', '%' -> 'percent')
    - Convert with the chosen phonetic algorithm (Soundex, Metaphone ...) to phonetic form (as pronounced in English)
    - Find similar, existing words (from unigrams file) based on matching suffixes (need to choose the appropriate suffix lenght)

- https://en.wikipedia.org/wiki/Soundex
    - (1918)
- https://en.wikipedia.org/wiki/Metaphone
    - Metaphone (1990)
    - Double Metaphone (2000)
    - Metaphone 3 (2009) - commercial product

**CMU ARPABET dictionary**

In [3]:
pronouncing.phones_for_word("sighs")

['S AY1 Z']

In [4]:
pronouncing.rhymes("failings")

['mailings', 'railings', 'tailings']

**Double Metaphone**

- does not take vowels (other than the first character) into account :(
- it can return both a primary and a secondary code for a string

In [5]:
def comp(t1, t2):
    print(f'{t1}\n{t2}')

In [6]:
comp(doublemetaphone("shell"), doublemetaphone("smell"))

('XL', '')
('SML', 'XML')


In [7]:
comp(doublemetaphone("generate"), doublemetaphone("degenerate"))

('JNRT', 'KNRT')
('TJNRT', 'TKNRT')


In [20]:
comp(doublemetaphone("mykola"), doublemetaphone("cocacola"))

('MKL', '')
('KKKL', '')


In [21]:
comp(doublemetaphone("mykole"), doublemetaphone("cocacola"))

('MKL', '')
('KKKL', '')


**Example "rhyme representation" algorithm**

- Input string s
- m = doublemetaphone(s)
- result = insert the vowels from s inside m  # (todo: how)
- in order to find set of rhymes match same suffixes # (todo: how long should the suffix be)

In [9]:
comp('generate',
     doublemetaphone('generate')[0])
'TJeNeRaTe'

generate
JNRT


'TJeNeRaTe'

In [10]:
comp('failings',
     doublemetaphone('failings')[0])
'FaiLiNKS'

failings
FLNKS


'FaiLiNKS'

In [12]:
comp("mykola", 
     doublemetaphone("mykola"))

'MyKoLa'

mykola
('MKL', '')


'MyKoLa'

In [16]:
comp("mass", 
     doublemetaphone("mass"))

'MaS'

mass
('MS', '')


'MaS'

**ARPABET model?**

In [1]:
!pip install g2p_en

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 13.3 MB/s eta 0:00:00 MB/s eta 0:00:0101
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 35.6 MB/s eta 0:00:00 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.3/180.3 kB 39.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 25.2 MB/s eta 0:00:00 MB/s eta 0:00:01
  Using cached click-8.1.3-py3-none-any.whl (96 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 769.7/769.7 kB 27.9 MB/s eta 0:00:00
  Using cached tqdm-4.65.0-py3-none-any.whl (77 kB)
  Created wheel for distance: filename=Distance-0.1.3-py3-none-any.whl size=16257 sha256=6855ca828b654137aa2675111fc8b255445fc58e4a09c938845dd3ad82ea7d3d
  Stored in directory: /home/byczong/.cache/pip/wheels/e8/bb/de/f71bf63559ea9a921059a5405806f7ff6ed612a9231c4a9309
Successfully built distance


In [30]:
from time import perf_counter
from g2p_en import G2p


g2p = G2p()

t1 = perf_counter()
out = g2p("qwertyuiop")
print(out)
t2 = perf_counter()

print(f'{t2-t1} s')

['K', 'W', 'ER1', 'T', 'IY0', 'AH0', 'W', 'AH0', 'M']
0.017724068999996234 s


In [23]:
pronouncing.search('N IH0 S T$')[:5]

['abortionist', 'africanist', 'agonist', 'anticommunist', 'arsonist']

In [25]:
pronouncing.search('AA1 L ER0 Z$')

['ahlers',
 'collars',
 "collor's",
 "controllers'",
 "dollar's",
 'dollars',
 "dollars'",
 'hollars',
 'hollers',
 "mahler's",
 "scholar's",
 'scholars',
 'sollars',
 'sollers',
 'wahlers',
 'zollars']

**Modify double metaphone to preserve vowels**